# Facilities

In [1]:
import pandas as pd
import re

In [2]:
facility = pd.read_excel("srv_ifmd_all-facilities.xlsx")

In [3]:
facility.columns

Index(['Facility ID', 'LGA Name', 'Facility Name', 'Facility_AutoNumber',
       'Street #', 'Street Name', 'Street Type', 'Suburb/Town', 'Pcode',
       'Melway Ref', 'VicRoads Ref', 'Latitude', 'Longitude',
       'Facility Ownership', 'Facility Purpose', 'Facility Category',
       'CFA Safer Place?', 'Access_', 'FaciltySportPlayedID', 'Facility ID.1',
       'Sports Played', 'Number of Field/Courts', 'Field/Surface Type',
       'Age of Facility', 'Condition of Facility', 'Facility Upgrade Age',
       'Changerooms', 'Facility Features',
       'Spectator numbers for seating/shelter', 'FullAddress',
       'MelwaysVicRoadsRef'],
      dtype='object')

## by ID

In [4]:
fy = facility[['LGA Name', 'Facility ID', 'Facility Name']]
fy = fy.drop_duplicates()
fy = fy.reset_index()
fy = fy[['LGA Name','Facility ID','Facility Name']]
fy

,LGA Name,Facility ID,Facility Name
0,NaN,MITCHE3674,Vermont South Club
1,Alpine Shire Council,ALPINE5519,Pioneer Park Recreation Reserve
2,Alpine Shire Council,ALPINE10930,Mount Beauty Pool & Stadium
3,Alpine Shire Council,ALPINE10928,Mount Beauty recreation Reserve
4,Alpine Shire Council,ALPINE10924,Dederang Recreation reserve
...,...,...,...
4995,Yarriambiack Shire Council,YARRIA12720,Murtoa Swimming Pool
4996,Yarriambiack Shire Council,YARRIA12730,Tempy Swimming Pool
4997,Yarriambiack Shire Council,YARRIA12735,Warracknabeal Memorial Swimming Pool
4998,Yarriambiack Shire Council,YARRIA12740,Woomelang Swimming Pool


In [5]:
fy.to_csv("facilities-byID.csv")

## by Council

In [6]:
fy

,LGA Name,Facility ID,Facility Name
0,NaN,MITCHE3674,Vermont South Club
1,Alpine Shire Council,ALPINE5519,Pioneer Park Recreation Reserve
2,Alpine Shire Council,ALPINE10930,Mount Beauty Pool & Stadium
3,Alpine Shire Council,ALPINE10928,Mount Beauty recreation Reserve
4,Alpine Shire Council,ALPINE10924,Dederang Recreation reserve
...,...,...,...
4995,Yarriambiack Shire Council,YARRIA12720,Murtoa Swimming Pool
4996,Yarriambiack Shire Council,YARRIA12730,Tempy Swimming Pool
4997,Yarriambiack Shire Council,YARRIA12735,Warracknabeal Memorial Swimming Pool
4998,Yarriambiack Shire Council,YARRIA12740,Woomelang Swimming Pool


In [7]:
missing = []
index = 0
for na in fy['LGA Name'].isna().values:
    if na: missing.append(index);
    index += 1;

In [8]:
missing

[0]

In [9]:
fy.iloc[missing]

,LGA Name,Facility ID,Facility Name
0,NaN,MITCHE3674,Vermont South Club


In [10]:
# based on the Sports and Recreation Facility ID
# the Bermont South Club is registered under 'Mitchell Shire Council'
fy['LGA Name'][missing] = 'Mitchell Shire Council'

In [11]:
fy

,LGA Name,Facility ID,Facility Name
0,Mitchell Shire Council,MITCHE3674,Vermont South Club
1,Alpine Shire Council,ALPINE5519,Pioneer Park Recreation Reserve
2,Alpine Shire Council,ALPINE10930,Mount Beauty Pool & Stadium
3,Alpine Shire Council,ALPINE10928,Mount Beauty recreation Reserve
4,Alpine Shire Council,ALPINE10924,Dederang Recreation reserve
...,...,...,...
4995,Yarriambiack Shire Council,YARRIA12720,Murtoa Swimming Pool
4996,Yarriambiack Shire Council,YARRIA12730,Tempy Swimming Pool
4997,Yarriambiack Shire Council,YARRIA12735,Warracknabeal Memorial Swimming Pool
4998,Yarriambiack Shire Council,YARRIA12740,Woomelang Swimming Pool


In [12]:
fyc = fy[['LGA Name','Facility ID']]

In [13]:
fyc = fyc.groupby('LGA Name').count().reset_index()
fyc

,LGA Name,Facility ID
0,Alpine Shire Council,6
1,Alpine Shire Council,31
2,Ararat Rural City Council,41
3,Ararat Rural City Council,2
4,Ballarat City Council,98
...,...,...
85,Wyndham City Council,61
86,Yarra City Council,109
87,Yarra Ranges Shire Council,96
88,Yarriambiack Shire Council,40


In [14]:
fyc.to_csv("facilities-byCouncil.csv")

## by LGA
Victoria has **79** councils, which means LGA Names have variations

In [15]:
fyc

,LGA Name,Facility ID
0,Alpine Shire Council,6
1,Alpine Shire Council,31
2,Ararat Rural City Council,41
3,Ararat Rural City Council,2
4,Ballarat City Council,98
...,...,...
85,Wyndham City Council,61
86,Yarra City Council,109
87,Yarra Ranges Shire Council,96
88,Yarriambiack Shire Council,40


In [16]:
count = dict()
c = 0
for i in fyc.values:
    i[0] = i[0].casefold()
    i[0] = re.sub('city council', '', i[0])
    i[0] = re.sub('shire council', '', i[0])
    i[0] = re.sub('rural', '', i[0])
    i[0] = i[0].strip()
    i[0] = i[0].title()
    if i[0] in count.keys():
        count[i[0]] += i[1]
    else:
        count[i[0]] = i[1]
        c+=1
        
print(c,' councils recorded')

79  councils recorded


In [17]:
count

{'Alpine': 37,
 'Ararat': 43,
 'Ballarat': 98,
 'Banyule': 74,
 'Bass Coast': 52,
 'Baw Baw': 93,
 'Bayside': 103,
 'Benalla': 20,
 'Bendigo': 135,
 'Boroondara': 88,
 'Borough Of Queenscliffe': 8,
 'Brimbank': 124,
 'Buloke': 48,
 'Campaspe': 71,
 'Cardinia': 72,
 'Casey': 106,
 'Central Goldfields': 36,
 'Colac Otway': 60,
 'Corangamite': 54,
 'Dandenong': 65,
 'Darebin': 61,
 'East Gippsland': 96,
 'Frankston': 89,
 'Gannawarra': 48,
 'Geelong': 183,
 'Glen Eira': 63,
 'Glenelg': 56,
 'Golden Plains': 49,
 'Hepburn': 34,
 'Hindmarsh': 26,
 'Hobsons Bay': 77,
 'Horsham': 43,
 'Hume': 75,
 'Indigo': 31,
 'Kingston': 95,
 'Knox': 99,
 'Latrobe': 95,
 'Loddon': 46,
 'Macedon Ranges': 60,
 'Manningham': 56,
 'Mansfield': 26,
 'Maribyrnong': 48,
 'Maroondah': 56,
 'Melbourne': 52,
 'Melton': 56,
 'Mildura': 54,
 'Mitchell': 50,
 'Moira': 65,
 'Monash': 97,
 'Moonee Valley': 65,
 'Moorabool': 37,
 'Moreland': 70,
 'Mornington Peninsula': 131,
 'Mount Alexander': 55,
 'Moyne': 40,
 'Murrind

In [18]:
facility_count = pd.DataFrame.from_dict(count, orient='index', columns=['Facility Count'])

In [19]:
facility_count.index.name = 'Govt Area'

In [20]:
facility_count.sort_values('Govt Area').reset_index()

,Govt Area,Facility Count
0,Alpine,37
1,Ararat,43
2,Ballarat,98
3,Banyule,74
4,Bass Coast,52
...,...,...
74,Wodonga,34
75,Wyndham,61
76,Yarra,109
77,Yarra Ranges,96


In [21]:
facility_count.to_csv("facilities-byGovt.csv")